In [37]:
sample = 'http://nos.ie/cultur/scannain/fisean-out-of-innocence-agallamh-le-heoin-o-dubhghaill/'

In [1]:
import requests
from bs4 import BeautifulSoup

In [40]:
page = requests.get(sample)

In [41]:
soup = BeautifulSoup(page.text, 'lxml')

In [5]:
vid = soup.find('div', {'id', 'video-wrapper'})

In [51]:
_get_video(soup)

'https://www.youtube.com/embed/lXr1QZPY7aY'

In [50]:
def _get_video(soup):
    vid = soup.find('div', {'id': 'video-wrapper'})
    if vid:
        iframe = vid.find('iframe')
        if iframe:
            return iframe.get('src', '')
    return ''

In [79]:
def _get_details(soup):
    details = {}
    pubdet = soup.find("div", {"id": "single-publish-details"})
    ptags = [p for p in pubdet.find_all('p')]
    if ptags[0].b:
        details['author'] = ptags[0].b.get_text(strip=True)
    if ptags[1]:
        details['date'] = ptags[1].get_text(strip=True)
    broll = pubdet.find("div", {"class": "blogroll-tag-category"})
    cats = set()
    for cat in broll.find_all("a", {"class": "featured-category"}):
        if cat.get_text(strip=True) != "":
            cats.add(cat.get_text(strip=True))
    if len(cats) > 0:
        details['categories'] = list(cats)

    tags = set()
    for tag in broll.find_all("a", {"class": "featured-tag"}):
        if tag.get_text(strip=True) != "":
            tags.add(tag.get_text(strip=True))
    if len(tags) > 0:
        details['tags'] = list(tags)
    return details

In [86]:
_get_subhead(soup)

''

In [81]:
def _get_subhead(soup):
    out = []
    content = soup.find("div", {"id": "single-area-center"})
    if content.h1 and content.h1.span:
        return content.h1.span.get_text(strip=True)
    else:
        return ''

In [83]:
def _mksoup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    return soup

In [2]:
def _read_menu():
    page = requests.get("http://nos.ie/")
    soup = BeautifulSoup(page.text, 'lxml')
    menu = soup.find("ul", {"id": "menu-main-menu"})
    cat_pages = set()
    for li in menu.find_all("li"):
        if li.a:
            cat_pages.add(li.a['href'])
    return cat_pages

In [5]:
links = _read_menu()

In [13]:
a = _get_article_list(links)

http://nos.ie/category/meain/raidio/page/7/
http://nos.ie/category/gniomhaiochas/comhshaol/page/8/
http://nos.ie/category/cultur/ceol/page/49/
http://nos.ie/category/saol/sport/page/11/
http://nos.ie/category/gniomhaiochas/teanga/page/26/
http://nos.ie/category/saol/slainte/page/7/
http://nos.ie/category/meain/page/28/
http://nos.ie/category/taisteal/page/8/
http://nos.ie/category/cultur/leabhair/page/10/
http://nos.ie/category/meain/soisialta/page/6/
http://nos.ie/category/cultur/dearadh/page/12/
http://nos.ie/category/cultur/scannain/page/14/
http://nos.ie/category/cultur/page/128/
http://nos.ie/category/gniomhaiochas/polaitiocht/page/8/
http://nos.ie/category/saol/stil/page/4/
http://nos.ie/category/saol/bia/page/8/
http://nos.ie/category/gniomhaiochas/page/49/
http://nos.ie/category/teic/page/14/
http://nos.ie/category/saol/page/57/
http://nos.ie/category/fiseain-nos/page/3/
http://nos.ie/category/cultur/ardan/page/6/
http://nos.ie/category/saol/craiceann/page/6/


In [14]:
len(a)

4138

In [11]:
len(a)

296

In [12]:
def _get_article_list(urls):
    rest = set()
    articles = set()
    for url in urls:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'lxml')
        new = _get_remainder(soup)
        rest = rest.union(new)
        art = _collect_articles(soup)
        articles = articles.union(art)
    for url in rest:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'lxml')
        art = _collect_articles(soup)
        articles = articles.union(art)
    return list(articles)        

In [3]:
def _get_remainder(soup):
    import re
    pagination = soup.find("div", {"class": "pagination"})
    if not pagination:
        return []
    current = pagination.find("span", {"class": "current"})
    if not (current and current.get_text(strip=True) == "1"):
        return []
    cats = [a for a in pagination.find_all('a')]
    last_cat = cats[-1]
    last_url = last_cat.get('href', '')
    if not last_url:
        return []
    print(last_url)
    m = re.match("(.*/)([0-9]+)/$", last_url)
    if not m:
        return []
    base = m.group(1)
    num = int(m.group(2)) + 1
    return [f'{base}{i}/' for i in range(2, num)]


In [9]:
def _collect_articles(soup):
    out = set()
    for art in soup.find_all("article", {"class": "blogroll-post"}):
        a = art.find('a')
        out.add(a.get('href'))
    return list(out)

In [17]:
top = _read_menu()

In [22]:
page = requests.get("http://nos.ie/category/cultur/ceol/")
soup = BeautifulSoup(page.text, 'lxml')
_collect_articles(soup)

['http://nos.ie/cultur/ceol/rianta-na-miosa-seinnliosta-nos-ar-spotify/',
 'http://nos.ie/cultur/ceol/na-tri-scor-bainte-amach-ag-an-amhranai-gaeltachta-enya/',
 'http://nos.ie/cultur/ceol/amhranai-conamaraioch-le-cloisteail-ar-amhran-nua-as-an-mbriotain/',
 'http://nos.ie/cultur/ceol/gradam-nos-buaite-ag-eabha-breathnach-ag-scleip-ghael-linn/',
 'http://nos.ie/cultur/ceol/lanquidity-sun-ra-agus-albaim-eile-na-miosa/',
 'http://nos.ie/cultur/ceol/cracks-agus-albaim-eile-na-miosa/',
 'http://nos.ie/cultur/ceol/cen-staisiun-raidio-ata-ar-fail-do-country-bumpkins-na-heireann/',
 'http://nos.ie/cultur/ceol/rianta-na-miosa-bealtaine-2/',
 'http://nos.ie/cultur/ceol/an-bhfuil-tu-ann-a-anam-amhran-nua-eisithe-ag-deorai/',
 'http://nos.ie/cultur/ceol/color-de-tropica-vol-2-agus-albaim-eile-na-miosa/',
 'http://nos.ie/cultur/ceol/boinn-oir-sheain-ui-riada-buaite-ag-fionnuala-donlon-agus-colm-broderick/',
 'http://nos.ie/cultur/ceol/rianta-na-miosa-marta-aibrean/',
 'http://nos.ie/cultur/rianta-

In [23]:
arts = _get_article_list(top)

In [24]:
arts

['http://nos.ie/cultur/scannain/cuig-scannan-is-fiu-a-fheiceail-an-samhradh-seo/',
 'http://nos.ie/cultur/cursai-oidhreachta-le-hoideas-gael-agus-imeachtai-eile/',
 'http://nos.ie/gniomhaiochas/comhshaol/ardu-croi-don-ghael-glas-soiniciuil-a-bhi-i-gcaint-michael-cronin-areir/',
 'http://nos.ie/gniomhaiochas/teanga/cainteoiri-bhearla-bhaile-atha-cliath-a-lorg-do-staidear/',
 'http://nos.ie/cultur/dearadh/go-raibh-maith-agaibh/',
 'http://nos.ie/saol/sport/is-matan-a-chaitear-a-aclu-an-intinn-chomh-maith/',
 'http://nos.ie/saol/bia/staidear-a-dheanamh-ag-driogaire-gaeltachta-ar-thionchar-na-haeraide-ar-dheantus-an-fhuisce/',
 'http://nos.ie/meain/raidio/gradam-buaite-ag-bladhaire-rnag-ag-gradaim-na-mean-ceilteach-areir/',
 'http://nos.ie/saol/sport/ta-bothar-fada-fos-le-siul-o-thaobh-meabhairshlainte-de-i-saol-an-spoirt/',
 'http://nos.ie/gniomhaiochas/polaitiocht/an-chead-diospoireacht-uachtaranachta-i-meiricea-thart-agus-2020-eirithe-nios-measa-aris/',
 'http://nos.ie/taisteal/ni-bhuai